# Exercise 6: SPARQL

In this exercise you will Learn how to use RDFLib to:
- Query a RDF Graph using SPARQL.

As a first step, let's import the python modules we need.

In [1]:
# pip install rdflib
import rdflib
from rdflib import Graph, URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, XSD, FOAF, DCTERMS, SDO, SKOS

## Querying with SPARQL

We will use:
- The following Turtle file (about Michael Jackson): https://www.wikidata.org/wiki/Special:EntityData/Q2831.ttl  
- The following resource (Michael Jackson): http://www.wikidata.org/entity/Q2831

In [2]:
g = Graph()
g.parse('https://www.wikidata.org/wiki/Special:EntityData/Q2831.ttl')

<Graph identifier=N457192cbdb0c43f68e6420e600d686ae (<class 'rdflib.graph.Graph'>)>

In [3]:
print(f'The graph g contains {len(g)} facts (triples)')

The graph g contains 23482 facts (triples)


In [4]:
# Get all objects with subject 'MJ' and predicate 'SKOS.altLabel'.
# Then print out the object's value and language 
MJ = URIRef('http://www.wikidata.org/entity/Q2831')
for label in g.objects(MJ, SKOS.altLabel):
    print(f'{label.value}@{label.language}')

Michael Joseph Jackson@de
MJ@de
Jacko@de
M.J.@de
M. J.@de
King of Pop@de
The King of Pop@de
Re del Pop@it
Michael Joseph Jackson@it
Jacko@it
Michael Joe Jackson@it
Michael J. Jackson@it
MJ@en
The King of Pop@en
Michael Joe Jackson@en
Michael Joseph Jackson@en
M.J.@en
M. J.@en
King of Pop@en
King of Music@en
Król Popu@pl
Michael Joseph Jackson@pl
Джексон, Майкл@ru
MJ@ru
The King of Pop@ru
Майкл Джозеф Джексон@ru
Король поп-музыки@ru
Майкл Джо Джексон@ru
Michael Joseph Jackson@nl
The King of Pop@nl
迈克尔杰克逊@zh
麥可·傑克森@zh
米高·積遜@zh
米高積遜@zh
杰克逊，M.@zh
MJ@gl
King Of Pop@gl
The King@gl
Michael Joseph Jackson@gl
MJ@es
Michael Joe Jackson@es
Michael Joseph Jackson@es
Rey del Pop@es
Rey de la Música@es
El Rey@es
米高·積遜@zh-hk
邁克爾傑克遜@zh-hant
Regele muzicii pop@ro
Regele pop-ului@ro
Michael Joseph Jackson@ro
A pop királya@hu
Jacko@hu
Michael Joseph Jackson@hu
MJ@hu
Michael Joseph Jackson@ca
MJ@ca
King Of Pop@ca
King of Music@ca
The King@ca
The King of Pop@lb
Michael Joseph Jackson@lb
MJ@lb
പോപ് രാജാവ്@m

In [5]:
# Do the same thing as above, but only get the preferred label for each object

# NOTE: The preferredLabel() method has been removed in newer versions of RDFLib

#labels = g.preferredLabel(MJ, labelProperties=(SKOS.altLabel,))
#for prop, label in labels:
#    print(f'{label.value}@{label.language}')

In [6]:
# Use a SPARQL query to:
# - Get all objects with subject 'wd:Q2831' and predicate 'skos:altLabel' (WHERE clause)
# - Print out all matched objects (SELECT clause)

qres = g.query('''
SELECT ?label
WHERE {
    wd:Q2831 skos:altLabel ?label .
}
''')

for label, *_ in qres:
    print(label.value, label.language)

Michael Joseph Jackson de
MJ de
Jacko de
M.J. de
M. J. de
King of Pop de
The King of Pop de
Re del Pop it
Michael Joseph Jackson it
Jacko it
Michael Joe Jackson it
Michael J. Jackson it
MJ en
The King of Pop en
Michael Joe Jackson en
Michael Joseph Jackson en
M.J. en
M. J. en
King of Pop en
King of Music en
Król Popu pl
Michael Joseph Jackson pl
Джексон, Майкл ru
MJ ru
The King of Pop ru
Майкл Джозеф Джексон ru
Король поп-музыки ru
Майкл Джо Джексон ru
Michael Joseph Jackson nl
The King of Pop nl
迈克尔杰克逊 zh
麥可·傑克森 zh
米高·積遜 zh
米高積遜 zh
杰克逊，M. zh
MJ gl
King Of Pop gl
The King gl
Michael Joseph Jackson gl
MJ es
Michael Joe Jackson es
Michael Joseph Jackson es
Rey del Pop es
Rey de la Música es
El Rey es
米高·積遜 zh-hk
邁克爾傑克遜 zh-hant
Regele muzicii pop ro
Regele pop-ului ro
Michael Joseph Jackson ro
A pop királya hu
Jacko hu
Michael Joseph Jackson hu
MJ hu
Michael Joseph Jackson ca
MJ ca
King Of Pop ca
King of Music ca
The King ca
The King of Pop lb
Michael Joseph Jackson lb
MJ lb
പോപ് രാജാവ് m

In [7]:
# Use a SPARQL query to:
# - Get all objects with subject 'wd:Q2831' and predicate
#   'rdfs:label' OR 'skos:prefLabel' OR 'skos:altLabel' AND
#   where the language of the object is 'en' (WHERE clause)
# - Print out all matched objects (SELECT clause)
# - Order by the objects' value ascending (ORDER BY clause)

qres = g.query('''
SELECT DISTINCT ?label
WHERE {
    wd:Q2831 rdfs:label | skos:prefLabel | skos:altLabel ?label
    FILTER( lang(?label) = 'en' )
}
ORDER BY ?label
''')

for label, *_ in qres:
    print(label)

King of Music
King of Pop
M. J.
M.J.
MJ
Michael Jackson
Michael Joe Jackson
Michael Joseph Jackson
The King of Pop


In [8]:
# Use a SPARQL query to:
# - Get all objects with subject 'wd:Q2831' and predicate 'wdt:P166', (1st part of WHERE clause)
#   where these matching objects are subjects with predicate 'rdfs:label', and retreive the objects for these (2nd part)
#   where the language of the object is 'en' (FILTER),
# - Also return the object's description if it has a 'schema:description' predicate (OPTIONAL).
# - Print out all matched objects and their description (SELECT clause)

qres = g.query('''
SELECT ?label ?description
WHERE {
    wd:Q2831 wdt:P166 ?award .
    ?award rdfs:label ?label .
    FILTER( lang(?label) = 'en' )
    OPTIONAL {
        ?award schema:description ?description
        FILTER( lang(?description) = 'en' )
    }
}
''')

for label, description in qres:
    print(f'Award: {label:<55} Description: {description}')

Award: Grammy Lifetime Achievement Award                       Description: music award given to individual performers or groups
Award: Grammy Legend Award                                     Description: special music award of merit given to recording artists
Award: International Artist Award of Excellence                Description: American Music Award
Award: Genesis Award                                           Description: awarded by the Humane Society of the United States
Award: Grammy Award for Best Male R&B Vocal Performance        Description: music award conferred between 1967 and 2011
Award: Grammy Award for Album of the Year                      Description: American music award for excellent albums
Award: Grammy Award for Best Male Pop Vocal Performance        Description: award
Award: Grammy Award for Record of the Year                     Description: award
Award: Grammy Award for Best Male Rock Vocal Performance       Description: Grammy Award
Award: Grammy Award for 